In [2]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as inputData
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta

In [4]:
data = inputData.read_data_sets('data', one_hot=True)

Extracting data\train-images-idx3-ubyte.gz
Extracting data\train-labels-idx1-ubyte.gz
Extracting data\t10k-images-idx3-ubyte.gz
Extracting data\t10k-labels-idx1-ubyte.gz


In [5]:
def plotNumbers(imset, valset, dimen):
    fig, axis = plt.subplots(dimen[0],dimen[1])

    for pos, sub in enumerate(axis.flat):
        sub.imshow(imset[pos].reshape((28,28)))
        sub.set_xticks([])
        sub.set_yticks([])
        sub.set_xlabel('pred:%r' %valset[pos])
    plt.show()

In [6]:
image_size = 28
input_size = 784
LRF_size = 5
cn1_filters = 12
cn2_filters = 20
output_layer = 10


X = tf.placeholder(tf.float32, [None, input_size])
X_for_conv = tf.reshape(X, [-1, image_size, image_size, 1])
Y_ori = tf.placeholder(tf.float32, shape=[None, output_layer])

Wcn1 = tf.Variable(initial_value=tf.truncated_normal(shape=[LRF_size, LRF_size, 1, cn1_filters], stddev=0.1))
bcn1 = tf.Variable(initial_value=tf.ones(shape=[cn1_filters])/10)

cn1 = tf.nn.conv2d(X_for_conv, Wcn1, strides=[1, 1, 1, 1], padding='SAME') + bcn1
cn1_relu = tf.nn.relu(cn1)

print('input is             %a' %X_for_conv)
print('weight is            %a' %Wcn1)
print('bias is              %a' %bcn1)
print('conv out, cn1 is     %a' %cn1)
print('conv relu, cn1_relu  %a' %cn1_relu)

input is             <tf.Tensor 'Reshape:0' shape=(?, 28, 28, 1) dtype=float32>
weight is            <tf.Variable 'Variable:0' shape=(5, 5, 1, 12) dtype=float32_ref>
bias is              <tf.Variable 'Variable_1:0' shape=(12,) dtype=float32_ref>
conv out, cn1 is     <tf.Tensor 'add:0' shape=(?, 28, 28, 12) dtype=float32>
conv relu, cn1_relu  <tf.Tensor 'Relu:0' shape=(?, 28, 28, 12) dtype=float32>


In [7]:
Wcn2 = tf.Variable(initial_value=tf.truncated_normal(shape=[LRF_size, LRF_size, cn1_filters, cn2_filters], stddev=0.1))
bcn2 = tf.Variable(initial_value=tf.ones(cn2_filters)/10)

cn2 = tf.nn.conv2d(cn1_relu, Wcn2, strides=[1, 1, 1, 1], padding='SAME') + bcn2
cn2_relu = tf.nn.relu(cn2)

print('weight is            %a' %Wcn2)
print('bias is              %a' %bcn2)
print('conv out, cn2 is     %a' %cn2)
print('conv relu, cn2_relu  %a' %cn2_relu)

weight is            <tf.Variable 'Variable_2:0' shape=(5, 5, 12, 20) dtype=float32_ref>
bias is              <tf.Variable 'Variable_3:0' shape=(20,) dtype=float32_ref>
conv out, cn2 is     <tf.Tensor 'add_1:0' shape=(?, 28, 28, 20) dtype=float32>
conv relu, cn2_relu  <tf.Tensor 'Relu_1:0' shape=(?, 28, 28, 20) dtype=float32>


In [8]:
fully_connected_input_size = cn2_relu.shape[1:4].num_elements()
cn2_reshaped = tf.reshape(cn2_relu, shape=[-1, fully_connected_input_size])

WL = tf.Variable(initial_value=tf.truncated_normal(shape=[fully_connected_input_size, output_layer], stddev=0.1)) 
bL = tf.Variable(initial_value=tf.ones(output_layer)/10)

logits_L = tf.matmul(cn2_reshaped, WL) + bL
Y_pred = tf.nn.softmax(logits_L)

print('cn2_reshaped is      %a' %cn2_reshaped)
print('weight is            %a' %WL)
print('bias is              %a' %bL)
print('logits_L is          %a' %logits_L)

cn2_reshaped is      <tf.Tensor 'Reshape_1:0' shape=(?, 15680) dtype=float32>
weight is            <tf.Variable 'Variable_4:0' shape=(15680, 10) dtype=float32_ref>
bias is              <tf.Variable 'Variable_5:0' shape=(10,) dtype=float32_ref>
logits_L is          <tf.Tensor 'add_2:0' shape=(?, 10) dtype=float32>


In [10]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits_L, labels=Y_ori)
cost = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer(learning_rate=0.003).minimize(cost)

correct_prediction = tf.equal(tf.argmax(Y_pred, axis=1), tf.argmax(Y_ori, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

accu_list = []
cost_list = []
iteration = 500
acc_check_list = np.arange(0, iteration, int(iteration/50))

start_time = time.time()
for val in range(iteration):
    next_batch_x, next_batch_y = data.train.next_batch(100)
    sess.run(optimizer, feed_dict={X: next_batch_x, Y_ori: next_batch_y})

    if val in acc_check_list:
        accu, cost_ = sess.run((accuracy, cost), feed_dict={X: data.test.images, Y_ori: data.test.labels})
        accu_list.append(accu)
        cost_list.append(cost_)
    
print(accu_list)

end_time = time.time()
print('total time taken is %s' %str(timedelta(seconds=int(round(end_time-start_time)))))

In [ ]:
plt.plot(acc_check_list, accu_list[:])
#plt.axis([0,iteration,0.88,0.99])

plt.plot(acc_check_list, cost_list[:])
plt.show()

In [ ]:
plt.plot(acc_check_list, accu_list[:])
plt.axis([0,iteration,0.88,0.99])
plt.show()

In [ ]:
li = ['a', 'a', 'b','b','b', 'b','c','d','d','d','d']
#one option
max(li,key=li.count)

#second option
from collections import Counter
data = Counter(li)
data.most_common(1)[0][0]

In [ ]:
[0.1046, 0.1355, 0.094, 0.1723, 0.1364, 0.1418, 0.1617, 0.1769, 0.1959, 0.2038, 0.1836, 0.1486, 0.1181, 0.1191, 0.1319, 0.1696, 
 0.2485, 0.3268, 0.374, 0.4423, 0.5107, 0.5764, 0.6159, 0.6456, 0.6565, 0.6398, 0.6516, 0.6938, 0.7298, 0.7433, 0.7533, 0.7492, 
 0.7586, 0.7717, 0.7808, 0.793, 0.8049, 0.8137, 0.8137, 0.8158, 0.824, 0.8233, 0.8199, 0.8233, 0.8244, 0.8206, 0.8317, 0.8373, 
 0.8427, 0.8449]
#total time taken is 0:12:08
#iterations:50

[0.1833, 0.2252, 0.2357, 0.2706, 0.314, 0.3577, 0.4375, 0.534, 0.6308, 0.6571, 0.7183, 0.7583, 0.7967, 0.8101, 0.8089, 0.82, 
 0.8463, 0.8467, 0.8631, 0.8755, 0.8743, 0.8791, 0.8843, 0.8938, 0.9012, 0.8997, 0.8933, 0.8968, 0.907, 0.9108, 0.9089, 0.9118, 
 0.9201, 0.9237, 0.9169, 0.9151, 0.9257, 0.9337, 0.9332, 0.9303, 0.9318, 0.9343, 0.9365, 0.9378, 0.9387, 0.9382, 0.9408, 0.943, 
 0.9421, 0.9458]
#total time taken is 0:12:22
#iterations:100